In [ ]:
import sys, os
from pyprojroot import here


# spyder up to find the root
# root = here(project_files=[".here"])
local = here(project_files=[".root"])

# append to path
# sys.path.append(str(root))
sys.path.append(str(local))

In [ ]:
import xarray as xr
import numpy as np
import requests as rq
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from tqdm.notebook import tqdm, trange
from kernellib.types import GeoData, Dimensions
from kernellib.kernels import SpatioTemporalKernel
from kernellib.preprocessing import create_oi_grid
from kernellib.dat.oi import reformat_oi_output
from kernellib.dat.l3 import read_l3_data
from kernellib.dat.l4 import read_l4_data
from kernellib.features.interp import interp_on_alongtrack
from kernellib.features.config import load_test_aoi
import hvplot.xarray
from pathlib import Path

%load_ext autoreload
%autoreload 2
# %load_ext memory_profiler

## Config

In [ ]:
from dataclasses import dataclass

@dataclass
class Params:
    # test data
    test_data_dir = "/bettik/johnsonj/data/data_challenges/ssh_mapping_2021/raw/netcdf"
    test_data_filename = "dt_gulfstream_c2_phy_l3_20161201-20180131_285-315_23-53.nc"
    aoi = load_test_aoi()
    
    # ref data
    ref_data_dir = "/bettik/johnsonj/data/data_challenges/ssh_mapping_2021/raw/netcdf"
    ref_data_filename = "mdt.nc"
    
    # results data
    results_dir = "/bettik/johnsonj/data/data_challenges/ssh_mapping_2021/interim"
    results_filename = "OSE_ssh_mapping_BASELINE.nc"
    
    # stats data
    stats_dir = "/bettik/johnsonj/data/data_challenges/ssh_mapping_2021/interim"
    
    

In [ ]:
from kernellib.pipeline.results import run_results_pipeline


In [ ]:
from omegaconf import OmegaConf

# init config file
config = Params()
# config = OmegaConf.structured(Params)

# run_results_pipeline(params)

In [ ]:
config

In [ ]:
from kernellib.dat.l3 import read_l3_data

# load test data
print("opening test dataset...")
ds_along_track = read_l3_data(
    Path(config.test_data_dir).joinpath(config.test_data_filename), 
    config.aoi
)

In [ ]:
# load oi results
print("opening oi results...")
ds_oi = xr.open_dataset(
    Path(config.results_dir).joinpath(config.results_filename)
)

In [ ]:
from kernellib.dat.oi import reformat_oi_output

# reformat oi results
print("reformatting oi results...")
ref_ds = xr.open_dataset(
    Path(config.ref_data_dir).joinpath(config.ref_data_filename)
)
ds_oi = reformat_oi_output(ds_oi, ref_ds)

In [ ]:
from kernellib.features.interp import interp_on_alongtrack

# interpolation
print("interpolating along track...")
time_alongtrack, lat_alongtrack, lon_alongtrack, ssh_alongtrack, ssh_map_interp = interp_on_alongtrack(
    ds_oi, 
    ds_along_track,
    aoi=config.aoi,
    is_circle=True
)

In [ ]:
from kernellib.utils import ResultsFileName

filenames = ResultsFileName(filename=config.results_filename)

In [ ]:
filenames.stats_filename

In [ ]:
%%time

from kernellib.types import RMSEBinning
from kernellib.features.stats.base import compute_stats

rmse_binning = RMSEBinning()

# stats config
leaderboard_nrmse, leaderboard_nrmse_std = compute_stats(
    time_alongtrack, 
    lat_alongtrack, 
    lon_alongtrack, 
    ssh_alongtrack, 
    ssh_map_interp, 
    rmse_binning.bin_lon_step,
    rmse_binning.bin_lat_step, 
    rmse_binning.bin_time_step,
    Path(config.results_dir).joinpath(f"stats_{config.results_filename}"),
    Path(config.results_dir).joinpath(f"stats_ts_{config.results_filename}"),
)

In [ ]:
from kernellib.viz.spatial import plot_spatial_statistics

plot_spatial_statistics(Path(config.results_dir).joinpath(f"stats_{config.results_filename}"))

In [ ]:
from kernellib.viz.temporal import plot_temporal_statistics

plot_temporal_statistics(
    Path(config.results_dir).joinpath(f"stats_ts_{config.results_filename}")
)

In [ ]:
t = str(Path(config.results_dir).joinpath(f"psd_{config.results_filename}"))
!rm $t

In [ ]:
from kernellib.types import SpectralStats
from kernellib.features.stats.spectral import compute_spectral_scores

psd_stats = SpectralStats()



compute_spectral_scores(
    time_alongtrack, 
    lat_alongtrack, 
    lon_alongtrack, 
    ssh_alongtrack, 
    ssh_map_interp, 
    psd_stats.length_scale,
    psd_stats.delta_x,
    psd_stats.delta_t,
    Path(config.results_dir).joinpath(f"psd_{config.results_filename}")
)

In [ ]:
from kernellib.viz.psd import plot_psd_score

plot_psd_score(Path(config.results_dir).joinpath(f"psd_{config.results_filename}"))

In [ ]:
# ds_oi_baseline.gssh_mu.hvplot.image(width=500, height=400, cmap="RdBu_r")

In [ ]:
# ds_oi_baseline.gssh_var.hvplot.image(width=500, height=400, cmap="viridis_r")

In [ ]:
# ds_oi_baseline.gssh_samples.hvplot.image("lat", "lon", width=500, height=400, cmap="RdBu_r")

## Leaderboard

In [ ]:
import pandas as pd
# Print leaderboard
data = [['BASELINE', 
         leaderboard_nrmse, 
         leaderboard_nrmse_std, 
         int(leaderboard_psds_score),
        'Covariances BASELINE OI',
        'example_eval_baseline.ipynb']]
Leaderboard = pd.DataFrame(data, 
                           columns=['Method', 
                                    "µ(RMSE) ", 
                                    "σ(RMSE)", 
                                    'λx (km)',  
                                    'Notes',
                                    'Reference'])
print("Summary of the leaderboard metrics:")
Leaderboard
print(Leaderboard.to_markdown())